In [1]:
import QuantLib as ql
import numpy as np
import pandas as pd
from datetime import datetime
from model_settings import ms
from itertools import product


pricing settings:
Actual/365 (Fixed) day counter
New York stock exchange calendar
compounding: continuous
frequency: annual



In [2]:
calculation_datetime = datetime.today()
calculation_date = ql.Date(
    calculation_datetime.day,
    calculation_datetime.month,
    calculation_datetime.year
)
r = 0.05
g = 0.02
s = 100.00
k = 100.00
w = 'put'
t = 360

v0 = 0.01 
kappa = 0.2
theta = 0.02
rho = -0.75
eta = 0.5

n_paths = 100000

past_fixings = 0 # Empty because this is a new contract

averaging_frequency = 30

## Geometric Asian Option

## Arithmetic Asian Option

# proprietary wrappers

In [3]:
my_geometric = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'geometric',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0,n_paths
            )
my_arithmetic = ms.ql_asian_price(
            s,k,t,r,g,calculation_datetime, w,
            'arithmetic',past_fixings,averaging_frequency,
            kappa,theta,rho,eta,v0,n_paths
            )

# Vanilla

In [4]:
vanilla = ms.ql_heston_price(
    s,k,t,r,g,w, 
    kappa, theta, rho, eta, v0, 
    calculation_datetime
)
print(f"\nvanilla: {vanilla}\n\ngeometric: {my_geometric}\n\narithmetic: {my_arithmetic}\n")


vanilla: 1.805832511155122

geometric: 1.7950527348380367

arithmetic: 1.800889588069121



# Data Generation

In [5]:
K = np.arange(int(s*0.5),int(s*1.5)+1,10).tolist()
averaging_frequencies = [5,10,15,30,60]
T = [60,180,360]
past_fixings = [0]




features = pd.DataFrame(
    product(
        [s],K,T,
        ['call','put'],
        ['geometric','arithmetic'],
        past_fixings,
        averaging_frequencies,
        [theta],
        [kappa],
        [rho],
        [eta],
        [v0],
    ),
    columns = [
        'spot_price',
        'strike_price',
        'days_to_maturity',
        'w',
        'averaging_type',
        'past_fixings',
        'averaging_frequency',
        'kappa','theta','rho','eta','v0',
    ]
)
features

,spot_price,strike_price,days_to_maturity,w,averaging_type,past_fixings,averaging_frequency,kappa,theta,rho,eta,v0
0,100.0,50,60,call,geometric,0,5,0.02,0.2,-0.75,0.5,0.01
1,100.0,50,60,call,geometric,0,10,0.02,0.2,-0.75,0.5,0.01
2,100.0,50,60,call,geometric,0,15,0.02,0.2,-0.75,0.5,0.01
3,100.0,50,60,call,geometric,0,30,0.02,0.2,-0.75,0.5,0.01
4,100.0,50,60,call,geometric,0,60,0.02,0.2,-0.75,0.5,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
655,100.0,150,360,put,arithmetic,0,5,0.02,0.2,-0.75,0.5,0.01
656,100.0,150,360,put,arithmetic,0,10,0.02,0.2,-0.75,0.5,0.01
657,100.0,150,360,put,arithmetic,0,15,0.02,0.2,-0.75,0.5,0.01
658,100.0,150,360,put,arithmetic,0,30,0.02,0.2,-0.75,0.5,0.01


In [6]:
help(ms.vector_asian_price)

Help on method vector_barrier_price in module model_settings.model_settings:

vector_barrier_price(df) method of model_settings.model_settings.model_settings instance
    Parameters
    ----------
    df : pandas dataframe
        df['spot_price'],
        df['strike_price'],
        df['days_to_maturity'],
        df['risk_free_rate'],
        df['dividend_rate'],
        df['calculation_date'],
        df['w'],
        df['barrier_type_name'],
        df['barrier'],
        df['rebate'],
        df['kappa'],
        df['theta'],
        df['rho'],
        df['eta'],
        df['v0']

    Returns
    -------
    barrier_prices : numpy array

